In [5]:
%cd ..
%ls



/home/aoezkan/eth/dslab/dsl-hs-2024/src
networks.py      pass_tables.py         resp.py                utils.py
notebooks/       __pycache__/           rl/
pass_options.py  responsibility.ipynb*  skillcorner_parser.py


In [9]:

import gym
# plot xt_table
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



In [ ]:

env = gym.make("LunarLander-v2")  # A popular Box2D environment
print(env)


<TimeLimit<OrderEnforcing<PassiveEnvChecker<LunarLander<LunarLander-v2>>>>>

In [4]:

# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)
for _ in range(100000):
    # this is where you would insert your policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [12]:
from pass_options import attacker_options
import pandas as pd


WYSCOUT_ID = 5414111
SKILLCORNER_ID = 952209


DATA_PATH = f'../data/networks/match_{SKILLCORNER_ID}/'
PASSES_DF_PATH = DATA_PATH + 'passes_df.csv'

DATA_PATH_PASSES = f'../data/'
XT_PLOT_PATH = DATA_PATH_PASSES + 'smoothed_xt.csv'


passes_df = pd.read_csv(PASSES_DF_PATH)


# load xt table
xt_table = pd.read_csv(XT_PLOT_PATH)

cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

# prepare with pitch_dict, outside the function
pitch_length = 105
pitch_width = 68
xt_rows, xt_cols = 68, 105
cell_width = pitch_length / xt_cols
cell_height = pitch_width / xt_rows

pitch_dict = {
    'pitch_length': pitch_length,
    'pitch_width': pitch_width,
    'xt_rows': xt_rows,
    'xt_cols': xt_cols,
    'cell_width': cell_width,
    'cell_height': cell_height,
    'xt_table': xt_table
}

print(pitch_dict['pitch_length'], pitch_dict['pitch_width'], pitch_dict['xt_rows'], pitch_dict['xt_cols'], pitch_dict['cell_width'], pitch_dict['cell_height'])

105 68 68 105 1.0 1.0


In [17]:
print(pitch_dict['pitch_length']/2, pitch_dict['pitch_width']/2)

print(passes_df['location.x'].max(), passes_df['location.x'].min())
print(passes_df['location.y'].max(), passes_df['location.y'].min())

52.5 34.0
49.75 -49.75
33.27157894736842 -31.814736842105265


In [23]:
frame_ids = passes_df['frame'].unique()

print(len(frame_ids))

# sample randomly frame from the frame_ids
np.random.choice(frame_ids)

893


16935

In [25]:
np.random.uniform(
            low=[0, 0], high=[10,20], size=(3, 2)
        ).astype(np.float32)

array([[ 4.0296807, 16.878597 ],
       [ 2.7674162,  2.7490032],
       [ 2.1063533,  1.0191805]], dtype=float32)